In [109]:
"""Neural Network for Santander
~~~~~~~~~~~~~~
We implement the stochastic gradient descent learning algorithm
for a feedforward neural network. ~4% of classes are 1s, 96% are 0s.
Improvements for class imbalance (outlier detection)
include the addition of the cross-entropy cost function,
regularization, and better initialization of network weights. 
"""

import json
import random
import sys
import numpy as np


#### Define the quadratic and cross-entropy cost functions

class QuadraticCost(object):

    @staticmethod
    def fn(a, y):
        """Return the cost associated with an output ``a`` and desired output
        ``y``.
        """
        penalty=14.5
        if y==1 and a < 0.5: fun = penalty*0.5*np.linalg.norm(a-y)**2
        else: fun = 1*0.5*np.linalg.norm(a-y)**2
        return fun

    @staticmethod
    def delta(z, a, y):
        """Return the error delta from the output layer."""
        return (a-y) * sigmoid_prime(z)


class CrossEntropyCost(object):

    @staticmethod
    def fn(a, y):
        """Return the cost associated with an output ``a`` and desired output
        ``y``.  Note that np.nan_to_num is used to ensure numerical
        stability.  In particular, if both ``a`` and ``y`` have a 1.0
        in the same slot, then the expression (1-y)*np.log(1-a)
        returns nan.  The np.nan_to_num ensures that that is converted
        to the correct value (0.0).
        """
        return np.sum(np.nan_to_num(-y*np.log(a)-(1-y)*np.log(1-a)))

    @staticmethod
    def delta(z, a, y):
        """Return the error delta from the output layer.  Note that the
        parameter ``z`` is not used by the method.  It is included in
        the method's parameters in order to make the interface
        consistent with the delta method for other cost classes.
        """
        return (a-y)


#### Network class
class Network(object):

    def __init__(self, sizes, cost=QuadraticCost):
        """ list ``sizes`` [a,b,c...] represents a multilayer network
        with b,c,d... hidden neurons in each layer.
        """
        self.num_layers = len(sizes)
        self.sizes = sizes
        self.default_weight_initializer()
        self.cost=cost

    def default_weight_initializer(self):
        """Initialize each weight using a Gaussian distribution with mean 0
        and standard deviation 1 over the ***square root of the number of
        weights*** connecting to the same neuron.  Initialize the biases
        using a Gaussian distribution with mean 0 and standard
        deviation 1. First layer has no biases. 
        """
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)/np.sqrt(x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def large_weight_initializer(self):
        """Initialize the weights using a Gaussian distribution with mean 0
        and standard deviation 1.  Initialize the biases using a
        Gaussian distribution with mean 0 and standard deviation 1.
        """
        self.biases = [np.random.randn(y, 1) for y in self.sizes[1:]]
        self.weights = [np.random.randn(y, x)
                        for x, y in zip(self.sizes[:-1], self.sizes[1:])]

    def feedforward(self, a):
        """Return the output of the network if ``a`` is input."""
        for b, w in zip(self.biases, self.weights):
            a = sigmoid(np.dot(w, a)+b)
        return a

    def SGD(self, training_data, epochs, mini_batch_size, eta,
            lmbda = 0.0,
            evaluation_data=None,
            monitor_evaluation_cost=False,
            monitor_evaluation_accuracy=False,
            monitor_training_cost=False,
            monitor_training_accuracy=False):
        """Train the neural network using mini-batch stochastic gradient
        descent.  The ``training_data`` is a list of tuples ``(x, y)``
        representing the training inputs and the desired outputs.  The
        other non-optional parameters are self-explanatory, as is the
        regularization parameter ``lmbda``.  The method also accepts
        ``evaluation_data``, usually either the validation or test
        data.  We can monitor the cost and accuracy on either the
        evaluation data or the training data, by setting the
        appropriate flags.  The method returns a tuple containing four
        lists: the (per-epoch) costs on the evaluation data, the
        accuracies on the evaluation data, the costs on the training
        data, and the accuracies on the training data.  All values are
        evaluated at the end of each training epoch.  So, for example,
        if we train for 30 epochs, then the first element of the tuple
        will be a 30-element list containing the cost on the
        evaluation data at the end of each epoch. 

        """
        if evaluation_data: n_data = len(evaluation_data)
        n = len(training_data)
        evaluation_cost, evaluation_accuracy = [], []
        training_cost, training_accuracy = [], []
        for j in xrange(epochs):
#             train_norm0 = [(x,y) for (x,y) in training_data if y==0]
#             train_norm1 = [(x,y) for (x,y) in training_data if y==1]
#             train_norm0 = random.sample(train_norm0, len(train_norm1))
#             training_data = train_norm0+train_norm1
            random.shuffle(training_data)
#             print sum([y==1 for (x,y) in training_data])
#             print n
#             print np.mean(test_data==1)
#             print np.mean((y==1 for (x,y) in training_data))
            mini_batches = [
                training_data[k:k+mini_batch_size]
                for k in xrange(0, len(training_data), mini_batch_size)]
            for mini_batch in mini_batches:
                self.update_mini_batch(
                    mini_batch, eta, lmbda, len(training_data))
            print "Epoch %s training complete" % j
            if monitor_training_cost:
                cost = self.total_cost(training_data, lmbda)
                training_cost.append(cost)
                print "Cost on training data: {}".format(cost)
            if monitor_training_accuracy:
                accuracy = self.accuracy(training_data)
                training_accuracy.append(accuracy)
                print "Accuracy on training data: {} / {}".format(
                    accuracy, n)
            if monitor_evaluation_cost:
                cost = self.total_cost(evaluation_data, lmbda)
                evaluation_cost.append(cost)
                print "Cost on evaluation data: {}".format(cost)
            if monitor_evaluation_accuracy:
                accuracy = self.accuracy(evaluation_data)
                evaluation_accuracy.append(accuracy)
                print "Accuracy on evaluation data: {} / {}".format(
                    self.accuracy(evaluation_data), n_data)
            print
        return evaluation_cost, evaluation_accuracy, \
            training_cost, training_accuracy

    def update_mini_batch(self, mini_batch, eta, lmbda, n):
        """Update the network's weights and biases by applying gradient
        descent using backpropagation to a single mini batch.  The
        ``mini_batch`` is a list of tuples ``(x, y)``, ``eta`` is the
        learning rate, ``lmbda`` is the regularization parameter, and
        ``n`` is the total size of the training data set.

        """
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        for x, y in mini_batch:
            delta_nabla_b, delta_nabla_w = self.backprop(x, y)
            weightsdict = {'0':1, '1':14.5}
            nabla_b = [nb + weightsdict[str(int(y))]*dnb for nb, dnb in zip(nabla_b, delta_nabla_b)]
            nabla_w = [nw + weightsdict[str(int(y))]*dnw for nw, dnw in zip(nabla_w, delta_nabla_w)]
        self.weights = [(1-eta*(lmbda/n))*w-(eta/len(mini_batch))*nw
                        for w, nw in zip(self.weights, nabla_w)]
        self.biases = [b-(eta/len(mini_batch))*nb
                       for b, nb in zip(self.biases, nabla_b)]

    def backprop(self, x, y):
        """Return a tuple ``(nabla_b, nabla_w)`` representing the
        gradient for the cost function C_x.  ``nabla_b`` and
        ``nabla_w`` are layer-by-layer lists of numpy arrays, similar
        to ``self.biases`` and ``self.weights``."""
        nabla_b = [np.zeros(b.shape) for b in self.biases]
        nabla_w = [np.zeros(w.shape) for w in self.weights]
        # feedforward
        activation = x
        activations = [x] # list to store all the activations, layer by layer
        zs = [] # list to store all the z vectors, layer by layer
        for b, w in zip(self.biases, self.weights):
            z = np.dot(w, activation)+b
            zs.append(z)
            activation = sigmoid(z)
            activations.append(activation)
        # backward pass
        delta = (self.cost).delta(zs[-1], activations[-1], y)
        nabla_b[-1] = delta
        nabla_w[-1] = np.dot(delta, activations[-2].transpose())
        # l = 1 means the last layer of neurons, l = 2 is the
        # second-last layer, and so on.
        for l in xrange(2, self.num_layers):
            z = zs[-l]
            sp = sigmoid_prime(z)
            delta = np.dot(self.weights[-l+1].transpose(), delta) * sp
            nabla_b[-l] = delta
            nabla_w[-l] = np.dot(delta, activations[-l-1].transpose())
        return (nabla_b, nabla_w)

    def accuracy(self, data):
        """Return the number of inputs in ``data`` for which the neural
        network outputs the correct result. The neural network's
        output is assumed to be the index of whichever neuron in the
        final layer has the highest activation.

        the program usually evaluates the cost
        on the training data and the accuracy on other data sets.

        """
        results = [(weightedround(self.feedforward(x)), y) for (x, y) in data]
        lst = [self.feedforward(x)[0][0] for (x, y) in data]
        print np.unique(lst)
            
        return sum(x == y for (x, y) in results)

    def total_cost(self, data, lmbda):
        """Return the total cost for data.
        """
        cost = 0.0
        for x, y in data:
            a = self.feedforward(x)
            if y == 1 and a < 0.5:
                cost += self.cost.fn(a, y)/len(data)
            else:
                cost += self.cost.fn(a, y)/len(data)
        cost += 0.5*(lmbda/len(data))*sum(np.linalg.norm(w)**2 for w in self.weights)
        return cost

    def save(self, filename):
        """Save the neural network to the file ``filename``."""
        data = {"sizes": self.sizes,
                "weights": [w.tolist() for w in self.weights],
                "biases": [b.tolist() for b in self.biases],
                "cost": str(self.cost.__name__)}
        f = open(filename, "w")
        json.dump(data, f)
        f.close()
        
        
    def predict(self, data):
        res = [self.feedforward(x)[0][0] for x in data]
        return res
    

#### Loading a Network
def load(filename):
    """Load a neural network from the file ``filename``.  Returns an
    instance of Network.

    """
    f = open(filename, "r")
    data = json.load(f)
    f.close()
    cost = getattr(sys.modules[__name__], data["cost"])
    net = Network(data["sizes"], cost=cost)
    net.weights = [np.array(w) for w in data["weights"]]
    net.biases = [np.array(b) for b in data["biases"]]
    return net

#### Other functions
def sigmoid(z):
    """The sigmoid function."""
    return 1.0/(1.0+np.exp(-z))

def sigmoid_prime(z):
    """Derivative of the sigmoid function."""
    return sigmoid(z)*(1-sigmoid(z))

def weightedround(x):
    return 0 if x<0.45 else 1


In [26]:
import pandas as pd
import numpy as np
from sklearn import preprocessing


train=pd.read_csv("/Users/bohun/Documents/DataSci/santandertrain.csv", sep=',')
train = train.set_index("ID")
x = train.values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
trainnorm = pd.DataFrame(x_scaled, columns = train.columns, index = train.index)
# trainnorm.to_csv("/Users/bohun/Documents/kaggleproject/trainnorm.csv", sep=',', encoding='utf-8')
labels = trainnorm['TARGET'].set_index("ID")

In [49]:
trainfilt = pd.read_csv("/Users/bohun/Documents/kaggleproject/filteredtrain.csv", sep=',',\
                        encoding='utf-8', dtype = np.float64, index_col = 0)
y = trainfilt.values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
y_scaled = min_max_scaler.fit_transform(y)
trainfilt = pd.DataFrame(y_scaled, columns = trainfilt.columns, index = trainfilt.index)
# trainfilt

In [55]:
import numpy as np
trainarrayfilt = trainfilt.values
trainarrayfilt = [np.reshape(x, (282, 1)) for x in trainarrayfilt]
training_datafilt = zip(trainarrayfilt, labels.values.T)

In [95]:
trainfilt = training_datafilt[0:55000]
testfilt = training_datafilt[55000:76020]

print len(testfilt), len(trainfilt)
print np.sum([j for (i,j) in trainfilt if j==1])
print np.sum([j for (i,j) in testfilt if j==1])


21020 55000
2129.0
879.0


In [37]:
import pandas as pd
trainnorm = pd.read_csv("/Users/bohun/Documents/kaggleproject/trainnorm.csv", sep=',', encoding='utf-8').set_index("ID")
trainarray2 = trainnorm.iloc[:,0:369].values #for svm

In [257]:
#balanced training set

# subset for TARGET = 1
train1 = trainnorm[trainnorm.TARGET == 1]
ind_train1 = random.sample(range(len(train1)), int(0.8*len(train1)))
ind_test1 = [i for i in range(len(train1)) if i not in ind_train1]
train1Train = train1.iloc[ind_train1, :]
train1Test = train1.iloc[ind_test1, :]

# subset for TARGET = 0
train0 = trainnorm[trainnorm.TARGET == 0]
ind_train0 = random.sample(range(len(train0)), int(0.8*len(train1)))
ind_test0 = random.sample([i for i in range(len(train0)) if i not in ind_train0],\
                          int(0.2*len(train1)))
train0Train = train0.iloc[ind_train0, :]
train0Test = train0.iloc[ind_test0, :]

trainnormsub = pd.concat((train0Train, train1Train), axis=0)
trainarray = trainnormsub.iloc[:,0:369].values
trainarray = [np.reshape(x, (369, 1)) for x in trainarray]
traintarget = trainnormsub.iloc[:,369].values.T

testnormsub = pd.concat([train0Test, train1Test], axis=0)
testarray = testnormsub.iloc[:,0:369].values
testarray = [np.reshape(x, (369, 1)) for x in testarray]
testtarget = testnormsub.iloc[:,369].values.T

test_data = zip(testarray, testtarget)
train_data = zip(trainarray, traintarget)

In [5]:
#imbalanced training, test set
import numpy as np
trainarray = trainnorm.iloc[:,0:369].values
trainarray = [np.reshape(x, (369, 1)) for x in trainarray]
target = trainnorm.iloc[:,369].values.T
training_data = zip(trainarray, target)
train_data = training_data
# test_data = training_data[60816:76020]

In [6]:
print np.mean([y==1 for (x,y) in train_data])
print np.mean([y==1 for (x,y) in test_data])
print np.unique(trainnormsub.TARGET)
print len(test_data)
print len(train_data)
print len(trainarray)
print len(testarray)

0.0395685345962


NameError: name 'test_data' is not defined

In [5]:
trainnorm.head()
print np.mean(target==1)
print trainnorm.shape
target

0.0395685345962
(76020, 370)


array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

In [107]:
trainnorm[trainnorm.TARGET == 1].shape

net.save("282100151etae3lbmda100.json")

In [103]:
net = Network([282, 100, 15, 1])
net.SGD(training_data=trainfilt, evaluation_data = testfilt, epochs=10, mini_batch_size=500, eta=1e-3, \
monitor_evaluation_cost = False, \
monitor_evaluation_accuracy = True, \
monitor_training_cost = True, \
monitor_training_accuracy = True, lmbda = 100)

Epoch 0 training complete
Cost on training data: 0.185117556407
[ 0.46590259  0.46597111  0.46597528 ...,  0.47460229  0.47472102
  0.47496223]
Accuracy on training data: 52871 / 55000
[ 0.46574669  0.46581586  0.46624725 ...,  0.47449484  0.47465987
  0.47467746]
[ 0.46574669  0.46581586  0.46624725 ...,  0.47449484  0.47465987
  0.47467746]
Accuracy on evaluation data: 20141 / 21020

Epoch 1 training complete
Cost on training data: 0.184182515664
[ 0.4597397   0.45978888  0.45982046 ...,  0.46841774  0.46855301
  0.46879041]
Accuracy on training data: 52871 / 55000
[ 0.45959135  0.45966344  0.46010062 ...,  0.46831782  0.4684911   0.4685153 ]
[ 0.45959135  0.45966344  0.46010062 ...,  0.46831782  0.4684911   0.4685153 ]
Accuracy on evaluation data: 20141 / 21020

Epoch 2 training complete
Cost on training data: 0.183359215228
[ 0.45396984  0.45400086  0.45405815 ...,  0.46262522  0.46277583
  0.46300967]
Accuracy on training data: 52871 / 55000
[ 0.45382864  0.45390344  0.4543458  ..

([],
 [20141, 20141, 20141, 20141, 20141, 20141, 20141, 20141, 20141, 20141],
 [0.18511755640728264,
  0.1841825156637468,
  0.18335921522820497,
  0.18263348627146672,
  0.18199329592532287,
  0.18142899428998968,
  0.18093111531960585,
  0.18049083029751198,
  0.18010233948423682,
  0.17975883739949297],
 [52871, 52871, 52871, 52871, 52871, 52871, 52871, 52871, 52871, 52871])

In [ ]:
# #  That's better! And so we can continue, individually adjusting each hyper-parameter,
# gradually improving performance. Once we've explored to find an improved value for ηη, 
# then we move on to find a good value for λλ. Then experiment with a more complex architecture,
# say a network with 10 hidden neurons. Then adjust the values for ηη and λλ again. Then increase to 20 hidden neurons. 
# And then adjust other hyper-parameters some more. And so on, at each stage evaluating performance using our 
# held-out validation data, and using those evaluations to find better and better hyper-parameters. As we do so, 
# it typically takes longer to witness the impact due to modifications of the hyper-parameters, and so we can 
# gradually decrease the frequency of monitoring.

In [212]:
trainarray2 = trainnorm.iloc[:,0:369].values
target = trainnorm.iloc[:,369].values.T
len(trainarray2)==len(target)

True

In [219]:
# #clf.support_vectors_
from sklearn import svm
trainarray2 = trainnorm.iloc[:,0:369].values
target = trainnorm.iloc[:,369].values.T
len(trainarray2)==len(target)


X = trainarray2[1000:10000]
y = target[1000:10000]
clf = svm.SVC(class_weight={1:20}, C=1.0, kernel = 'rbf')
clf.fit(X, y)  
# SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
#     decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
#     max_iter=-1, probability=False, random_state=None, shrinking=True,
#     tol=0.001, verbose=False)
len(clf.support_vectors_)
np.mean([clf.predict(trainarray2[10000:12000])==target[10000:12000]])

0.71299999999999997

In [248]:
trainarray2 = trainnorm.iloc[:,0:369].values
print np.mean(target == 1)
trainarray2.shape

0.0395685345962


(76020, 369)

In [207]:
from sklearn import svm
import matplotlib.pyplot as plt


X = trainarray2[1:1000]
y = target[1:1000]
clf = svm.SVC(class_weight={1:2}, C=3, kernel = 'linear')
clf.fit(X, y) 
print len(clf.support_vectors_)
clf.predict()

156

In [46]:
# [(np.around(net.feedforward(train_data[1][0]) ), y) for (x, y) in train_data]
#clf.coef_
np.linspace(-5, 5,3)

array([-5.,  0.,  5.])

In [9]:
trainarray2[1:100].shape

(99, 369)

In [249]:
results = [(round(net.feedforward(train_data[1][0])), y) for (x, y) in train_data]

sum(x == y for (x, y) in results)

58468

In [232]:
net.feedforward(train_data[2][0])

array([[ 0.21667452]])

In [90]:
round(np.array([1]))

1.0

In [133]:
(5269.4576654855)

5269.4576654855